In [192]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

In [2]:
def rest(df):
    game = np.array(df['G'].fillna(0))
    rested = game == 0
    df['Rest'] = rested*1
    return df

def str_to_int(s):
    if type(s) == int or type(s) == float:
        return s
    if len(re.findall('\d', s)) != 0:
        if '.' in s:
            return float(s)
        return int(s[:2])
    return 0

def min_to_int(s):
    if len(re.findall('\d', s)) != 0:
        return int(s.split(':')[0])
    return 0

def plus_min(s):
    if type(s) == int or type(s) == float:
        return s
    if len(re.findall('\d', s)) == 0:
        return 0
    if s[0] == '-':
        return int(s[1:]) * -1
    if s[0] == '+':
        return int(s[1:]) 
    return 0

def meaner(df, col):
    col_vals = df[col].apply(str_to_int).values
    length = len(col_vals)
    df[col] = np.cumsum(col_vals)/(np.arange(length) + 1)
    return df

def meaner_min(df, col):
    col_vals = df[col].apply(min_to_int).values
    length = len(col_vals)
    df[col] = np.cumsum(col_vals)/(np.arange(length) + 1)
    return df

def meaner_int(df, col):
    col_vals = df[col].values
    length = len(col_vals)
    df[col] = np.cumsum(col_vals)/(np.arange(length) + 1)
    return df

def games(df, col):
    game = df[col].fillna(0)
    length = len(game)
    new_game = []
    for i in np.arange(length):
        new_game.append(max(game[:(i + 1)]))
    df[col] = new_game
    return df

def manipulate(df, inj, player):
    df = df.drop(labels=['Tm', 'GS', 'Unnamed: 5', 'Opp', 'PF', 'Unnamed: 7'], axis=1)
    df = rest(df) 
    df['Age'] = df['Age'].apply(str_to_int) 
    
    df['+/-'] = df['+/-'].apply(plus_min) 
    
    df = games(df, 'G')
    df = meaner_min(df, 'MP')
    df = meaner(df, 'FG')
    df = meaner(df, 'FGA')
    df = meaner(df, '3P')
    df = meaner(df, '3PA')
    df = meaner(df, 'FT')
    df = meaner(df, 'FTA')
    
    df['FG%'] = df['FG']/df['FGA']
    df['3P%'] = df['3P']/df['3PA']
    df['FT%'] = df['FT']/df['FTA']
    
    df = meaner(df, 'ORB') 
    df = meaner(df, 'DRB')
    df = meaner(df, 'TRB')
    df = meaner(df, 'AST')
    df = meaner(df, 'STL')
    df = meaner(df, 'BLK')
    df = meaner(df, 'TOV')
    df = meaner(df, 'PTS')
    df = meaner(df, 'GmSc')
    
    df = meaner_int(df, '+/-')
    
    player_i = inj[inj['Relinquised'] == player]
    player_i = player_i[['Date', 'score']]
    
    df = df.merge(player_i, how='left')
    df['score'] = df['score'].fillna(0)
    df['score'] = np.cumsum(df['score'].values)
    df = df.drop('Date', axis=1)
    return df

In [241]:
i = pd.read_csv('injury_with_score.csv')
# i.loc[1719, 'Date'] = '2017-02-28'
# i.loc[2799, 'Date'] = '2018-03-06'
# i.loc[2813, 'Date'] = '2018-03-10'
# i.loc[2353, 'Date'] = '2017-12-06'
i.loc[2667, 'Date'] = '2018-01-30'

In [242]:
def conc(files, names):
    zer = pd.read_csv(files[0])
    one = pd.read_csv(files[1])
    two = pd.read_csv(files[2])
    thr = pd.read_csv(files[3])
    fou = pd.read_csv(files[4])
    fiv = pd.read_csv(files[5])
    six = pd.read_csv(files[6])
    sev = pd.read_csv(files[7])
    eig = pd.read_csv(files[8])
    nin = pd.read_csv(files[9])
    
    nzer = manipulate(zer, i, names[0])
    none = manipulate(one, i, names[1])
    ntwo = manipulate(two, i, names[2])
    nthr = manipulate(thr, i, names[3])
    nfou = manipulate(fou, i, names[4])
    nfiv = manipulate(fiv, i, names[5])
    nsix = manipulate(six, i, names[6])
    nsev = manipulate(sev, i, names[7])
    neig = manipulate(eig, i, names[8])
    nnin = manipulate(nin, i, names[9])
    
    team = pd.concat([nzer, none, ntwo, nthr, nfou, 
                     nfiv, nsix, nsev, neig, nnin])
    team.drop('Rk', axis=1, inplace=True)
#     team.drop('iRk', axis=1, inplace=True)
    team = team.reset_index()
    team.drop('index', axis=1, inplace=True)
    return team

In [247]:
lst = "hawks17-18.csv heat17-18.csv hornets17-18.csv magic17-18.csv thunder17-18.csv wizards17-18.csv"
lst.split(' ')

['hawks17-18.csv',
 'heat17-18.csv',
 'hornets17-18.csv',
 'magic17-18.csv',
 'thunder17-18.csv',
 'wizards17-18.csv']

In [244]:
files = ['bradley_beal.csv',
 'ian_mahinmi.csv',
 'john_wall.csv',
 'kelly_oubre.csv',
 'marcin_gortat.csv',
 'markieff_morris.csv',
 'mike_scott.csv',
 'otto_porter.csv',
 'ramon_sessions.csv',
 'tomas_satoransky.csv']
names = ['Bradley Beal', 'Ian Mahinmi', 'John Wall (a)', 'Kelly Oubre', 'Marcin Gortat', 'Markieff Morris', 'Mike Scott',
        'Otto Porter Jr.', 'Ramon Sessions', 'Tomas Satoransky']
path = "wizards17-18/"
files = [path + x for x in files] 

team = conc(files, names)

In [245]:
team.to_csv('logs17-18/wizards17-18.csv', index = None, header=True)

In [246]:
set(team['score'])

{0.0, 1.0, 2.0, 9.0, 10.0}

In [19]:
i = pd.read_csv("injury_with_score.csv")

In [221]:
x = i[i.Team == 'Wizards']

In [222]:
set(x.Relinquised)

{'Alan Anderson',
 'Bradley Beal',
 'Daniel Ochefu',
 'Danuel House',
 'DeJuan Blair',
 'Drew Gooden',
 'Ian Mahinmi',
 'Jason Smith',
 'John Wall (a)',
 'Kelly Oubre',
 'Marcin Gortat',
 'Markieff Morris',
 'Mike Scott',
 'Nene / Nene Hilario / Maybyner Hilario',
 'Otto Porter Jr.',
 'Sheldon McClellan / Sheldon Mac',
 'Tim Frazier',
 'Tomas Satoransky',
 nan}

In [240]:
i[i['Relinquised'] == 'Ian Mahinmi']

,Date,Team,Acquired,Relinquised,injury,severity,score
137,2016-01-17,Pacers,NaN,Ian Mahinmi,sore left heel,DTD,1
179,2016-01-22,Pacers,NaN,Ian Mahinmi,sprained left ankle,DTD,1
242,2016-01-30,Pacers,NaN,Ian Mahinmi,sore lower back,DTD,1
893,2016-10-15,Wizards,NaN,Ian Mahinmi,surgery on left knee to repair torn medial men...,out indefinitely,8
2390,2017-12-12,Wizards,NaN,Ian Mahinmi,sore right knee,DTD,1
3014,2018-04-06,Wizards,NaN,Ian Mahinmi,concussion,DTD,1


In [238]:
pd.read_csv('wizards17-18/mike_scott.csv')

,Rk,G,Date,Age,Tm,Unnamed: 5,Opp,Unnamed: 7,GS,MP,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,1,1.0,2017-10-18,29-094,WAS,NaN,PHI,W (+5),0,14:50,...,2,2,0,0,0,1,0,7,3.6,-9
1,2,2.0,2017-10-20,29-096,WAS,NaN,DET,W (+4),0,20:42,...,0,0,1,0,1,0,0,8,7.8,-10
2,3,3.0,2017-10-23,29-099,WAS,@,DEN,W (+5),0,17:44,...,1,1,1,0,0,2,2,2,-2.9,-3
3,4,4.0,2017-10-25,29-101,WAS,@,LAL,L (-3),0,25:28,...,8,8,0,1,0,0,2,4,0.0,-7
4,5,5.0,2017-10-27,29-103,WAS,@,GSW,L (-3),0,24:03,...,6,7,1,0,0,2,3,14,10.7,-1
5,6,6.0,2017-10-29,29-105,WAS,@,SAC,W (+27),0,15:07,...,1,4,0,0,0,1,1,13,10.1,+18
6,7,7.0,2017-11-01,29-108,WAS,NaN,PHO,L (-6),0,32:30,...,6,8,1,0,0,0,2,12,9.4,-1
7,8,8.0,2017-11-03,29-110,WAS,NaN,CLE,L (-8),0,17:19,...,3,3,0,0,0,1,5,12,7.0,-5
8,9,9.0,2017-11-05,29-112,WAS,@,TOR,W (+11),0,16:54,...,3,3,0,1,1,2,5,5,0.8,0
9,10,10.0,2017-11-07,29-114,WAS,NaN,DAL,L (-14),0,7:32,...,2,2,0,0,0,0,0,2,1.9,-9


In [252]:
path2 = 'logs17-18/'
files2 = ['hawks17-18.csv',
 'heat17-18.csv',
 'hornets17-18.csv',
 'magic17-18.csv',
 'thunder17-18.csv',
 'wizards17-18.csv']
files2 = [path2 + x for x in files2]

In [257]:
concat = []
for file in files2: 
    concat.append(pd.read_csv(file))
sev_eig = pd.concat(concat)
sev_eig = sev_eig.reset_index()
sev_eig.drop('index', axis=1, inplace=True)
sev_eig

/Users/benjaminlee/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


,+/-,3P,3P%,3PA,AST,Age,BLK,DRB,FG,FG%,...,GmSc,MP,ORB,PTS,Rest,Rk,STL,TOV,TRB,score
0,9.000000,2.000000,0.500000,4.000000,2.000000,25,0.000000,4.000000,4.000000,0.500000,...,10.800000,17.000000,0.000000,13.000000,0,1.0,1.000000,1.000000,4.000000,0.0
1,6.000000,2.500000,0.555556,4.500000,1.500000,25,0.000000,3.000000,4.500000,0.562500,...,11.550000,21.500000,0.000000,13.500000,0,2.0,1.000000,0.500000,3.000000,0.0
2,4.000000,2.000000,0.461538,4.333333,1.333333,25,0.000000,2.333333,3.666667,0.458333,...,8.466667,21.000000,0.333333,11.333333,0,3.0,1.000000,0.666667,2.666667,0.0
3,4.500000,1.500000,0.400000,3.750000,1.250000,25,0.000000,3.250000,3.250000,0.433333,...,7.275000,23.500000,0.250000,9.750000,0,4.0,0.750000,0.500000,3.500000,0.0
4,5.000000,1.200000,0.375000,3.200000,1.800000,25,0.000000,2.600000,3.400000,0.447368,...,8.580000,23.200000,0.600000,10.400000,0,5.0,1.200000,0.800000,3.200000,0.0
5,3.500000,1.333333,0.363636,3.666667,1.666667,25,0.000000,3.500000,3.500000,0.437500,...,8.366667,24.333333,0.500000,10.333333,0,6.0,1.166667,0.833333,4.000000,0.0
6,1.285714,1.142857,0.296296,3.857143,1.571429,25,0.000000,3.714286,3.285714,0.403509,...,7.100000,24.428571,0.428571,9.428571,0,7.0,1.142857,0.857143,4.142857,0.0
7,0.375000,1.000000,0.275862,3.625000,2.000000,25,0.125000,3.750000,3.250000,0.412698,...,7.787500,25.125000,0.375000,9.250000,0,8.0,1.375000,0.750000,4.125000,0.0
8,-1.111111,1.111111,0.285714,3.888889,1.888889,25,0.111111,3.777778,3.777778,0.425000,...,8.522222,25.888889,0.333333,10.444444,0,9.0,1.444444,0.666667,4.111111,0.0
9,-2.300000,1.000000,0.270270,3.700000,1.900000,25,0.100000,3.700000,3.600000,0.413793,...,7.760000,25.600000,0.400000,9.800000,0,10.0,1.300000,0.800000,4.100000,0.0


In [100]:
# concat = []
# for file in files2: 
#     concat.append(pd.read_csv(file))
# six_sev = pd.concat(concat)
# six_sev = six_sev.reset_index()
# six_sev.drop('index', axis=1, inplace=True)
# six_sev

,G,Age,MP,FG,FGA,FG%,3P,3PA,3P%,FT,...,TRB,AST,STL,BLK,TOV,PTS,GmSc,+/-,Rest,score
0,1.0,23,25.000000,6.000000,12.000000,0.500000,2.000000,4.000000,0.500000,0.000000,...,2.000000,2.000000,1.000000,0.000000,4.000000,14.000000,5.800000,-1.000000,0,0.0
1,2.0,23,26.000000,5.000000,9.500000,0.526316,1.000000,2.500000,0.400000,1.500000,...,2.500000,6.500000,1.000000,0.000000,3.000000,12.500000,10.350000,5.500000,0,0.0
2,3.0,23,28.000000,5.000000,11.000000,0.454545,1.333333,3.333333,0.400000,2.666667,...,2.666667,6.000000,0.666667,0.000000,2.333333,14.000000,11.100000,7.333333,0,0.0
3,4.0,23,28.000000,4.750000,11.000000,0.431818,1.250000,3.250000,0.384615,2.500000,...,2.500000,6.000000,0.750000,0.000000,2.250000,13.250000,10.300000,2.750000,0,0.0
4,5.0,23,28.800000,5.200000,12.000000,0.433333,1.000000,3.200000,0.312500,3.200000,...,3.400000,5.400000,0.600000,0.000000,2.600000,14.600000,10.440000,1.200000,0,0.0
5,6.0,23,29.333333,5.333333,12.333333,0.432432,1.166667,3.500000,0.333333,3.166667,...,3.333333,6.166667,0.666667,0.000000,2.833333,15.000000,10.983333,1.500000,0,0.0
6,7.0,23,29.714286,6.000000,12.857143,0.466667,1.428571,3.857143,0.370370,3.428571,...,3.142857,6.142857,0.571429,0.000000,3.000000,16.857143,12.385714,1.714286,0,0.0
7,8.0,23,29.875000,5.875000,13.625000,0.431193,1.375000,3.875000,0.354839,3.250000,...,3.625000,5.750000,0.500000,0.000000,3.125000,16.375000,11.025000,-0.625000,0,0.0
8,9.0,23,29.555556,6.000000,13.222222,0.453782,1.444444,3.666667,0.393939,2.888889,...,3.777778,6.000000,0.555556,0.111111,3.333333,16.333333,11.400000,0.444444,0,0.0
9,10.0,23,29.800000,6.100000,13.400000,0.455224,1.500000,3.500000,0.428571,2.800000,...,3.500000,5.800000,0.600000,0.100000,3.500000,16.500000,11.200000,0.400000,0,0.0


In [255]:
sev_eig.to_csv('ben17-18logs.csv', index = None, header=True)